PROBLEM STATEMENT

Dataset (attached with the task): The data contains a pair of paragraphs. These text paragraphs are randomly sampled from a raw dataset. Each pair of sentences may or may not be semantically similar. The candidate is to predict a value between 0-1 indicating the similarity between the pair of text paras. A sample of a similar dataset will be used as test data, therefore it’s crucial to the model solution using provided dataset.

Part A
Build an algorithm/model that can quantify the degree of similarity between the two text-based on Semantic similarity. Semantic Textual Similarity (STS) assesses the degree to which two sentences are semantically equivalent to each other.
1 means highly similar, 0 means highly dissimilar.

IMPORT SOME LIBRARIES

In [1]:
import nltk
nltk.download('stopwords')
nltk.download("word_tokenize")
nltk.download('wordnet')
nltk.download()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shabnam/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading word_tokenize: Package 'word_tokenize' not
[nltk_data]     found in index
[nltk_data] Downloading package wordnet to /Users/shabnam/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [2]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats


# importing libraries 
from nltk.tokenize import word_tokenize,sent_tokenize
from string import punctuation
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer,LancasterStemmer
#from contractions import fix 
#from unidecode import unidecode
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB,BernoulliNB,GaussianNB
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import numpy as np 
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV, StratifiedKFold
import pandas
from tqdm import tqdm
tqdm.pandas()

from gensim.models import Word2Vec,doc2vec

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans

from sklearn.svm import SVC,SVR

import warnings
warnings.filterwarnings('ignore')
import re
import gensim
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec, KeyedVectors

READ DATASET

In [3]:
path='/Users/shabnam/Downloads/DataNeuron_DataScience_Task1/untitled folder/DataNeuron_Text_Similarity.csv'
df=pd.read_csv(path)
df.head()

,text1,text2
0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...
1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...
2,player burn-out worries robinson england coach...,hanks greeted at wintry premiere hollywood sta...
3,hearts of oak 3-2 cotonsport hearts of oak set...,redford s vision of sundance despite sporting ...
4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens with victory in la amelie maure...


In [4]:
df.shape

(3000, 2)

In [5]:
df.isnull().sum()

text1    0
text2    0
dtype: int64

PRE-PROCESSING DATASET
Convert phrases like won't to will not using function decontracted() below
Remove Stopwords.
Remove any special symbols and lower case all words
lemmatizing words using WordNetLemmatizer define in function word_tokenizer below

In [6]:
def decontract(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [7]:
# Combining all the above stundents 

preprocessed_text1 = []

# tqdm is for printing the status bar

for sentance in tqdm(df['text1'].values):
    sent = decontract(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)

    sent = ' '.join(e for e in sent.split() if e not in stopwords.words('english'))
    preprocessed_text1.append(sent.lower().strip())

  0%|          | 0/3000 [00:00<?, ?it/s]

100%|██████████| 3000/3000 [01:52<00:00, 26.57it/s]


In [8]:

from nltk.tokenize import word_tokenize
df['text1'] = df['text1'].apply(word_tokenize)

df

,text1,text2
0,"[broadband, challenges, tv, viewing, the, numb...",gardener wins double in glasgow britain s jaso...
1,"[rap, boss, arrested, over, drug, find, rap, m...",amnesty chief laments war failure the lack of ...
2,"[player, burn-out, worries, robinson, england,...",hanks greeted at wintry premiere hollywood sta...
3,"[hearts, of, oak, 3-2, cotonsport, hearts, of,...",redford s vision of sundance despite sporting ...
4,"[sir, paul, rocks, super, bowl, crowds, sir, p...",mauresmo opens with victory in la amelie maure...
...,...,...
2995,"[uk, directors, guild, nominees, named, martin...",steel firm to cut 45 000 jobs mittal steel ...
2996,"[u2, to, play, at, grammy, awards, show, irish...",israel looks to us for bank chief israel has a...
2997,"[pountney, handed, ban, and, fine, northampton...",india and iran in gas export deal india has si...
2998,"[belle, named, best, scottish, band, belle, &,...",mido makes third apology ahmed mido hossam h...


In [9]:
lm = WordNetLemmatizer()
df['text1']=df['text1'].apply(lambda row: [lm.lemmatize(word) for word in row])

df

,text1,text2
0,"[broadband, challenge, tv, viewing, the, numbe...",gardener wins double in glasgow britain s jaso...
1,"[rap, bos, arrested, over, drug, find, rap, mo...",amnesty chief laments war failure the lack of ...
2,"[player, burn-out, worry, robinson, england, c...",hanks greeted at wintry premiere hollywood sta...
3,"[heart, of, oak, 3-2, cotonsport, heart, of, o...",redford s vision of sundance despite sporting ...
4,"[sir, paul, rock, super, bowl, crowd, sir, pau...",mauresmo opens with victory in la amelie maure...
...,...,...
2995,"[uk, director, guild, nominee, named, martin, ...",steel firm to cut 45 000 jobs mittal steel ...
2996,"[u2, to, play, at, grammy, award, show, irish,...",israel looks to us for bank chief israel has a...
2997,"[pountney, handed, ban, and, fine, northampton...",india and iran in gas export deal india has si...
2998,"[belle, named, best, scottish, band, belle, &,...",mido makes third apology ahmed mido hossam h...


In [10]:
# Merging preprocessed_text1 in text_data
df['text1'] = preprocessed_text1

df.head(3)

,text1,text2
0,broadband challenges tv viewing number europea...,gardener wins double in glasgow britain s jaso...
1,rap boss arrested drug find rap mogul marion s...,amnesty chief laments war failure the lack of ...
2,player burn worries robinson england coach and...,hanks greeted at wintry premiere hollywood sta...


In [11]:
# Combining all the above stundents 
from tqdm import tqdm
preprocessed_text2 = []

# tqdm is for printing the status bar
for sentance in tqdm(df['text2'].values):
    sent = decontract(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    
   
    sent = ' '.join(e for e in sent.split() if e not in stopwords.words('english'))
    preprocessed_text2.append(sent.lower().strip())

100%|██████████| 3000/3000 [02:04<00:00, 24.16it/s]


In [12]:
# Merging preprocessed_text2 in text_data

df['text2'] = preprocessed_text2
df.head()

,text1,text2
0,broadband challenges tv viewing number europea...,gardener wins double glasgow britain jason gar...
1,rap boss arrested drug find rap mogul marion s...,amnesty chief laments war failure lack public ...
2,player burn worries robinson england coach and...,hanks greeted wintry premiere hollywood star t...
3,hearts oak 3 2 cotonsport hearts oak set ghana...,redford vision sundance despite sporting cordu...
4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens victory la amelie mauresmo mari...


In [13]:
df['text2'] = df['text2'].apply(word_tokenize)
df['text2']=df['text2'].apply(lambda row: [lm.lemmatize(word) for word in row])
df

,text1,text2
0,broadband challenges tv viewing number europea...,"[gardener, win, double, glasgow, britain, jaso..."
1,rap boss arrested drug find rap mogul marion s...,"[amnesty, chief, lament, war, failure, lack, p..."
2,player burn worries robinson england coach and...,"[hank, greeted, wintry, premiere, hollywood, s..."
3,hearts oak 3 2 cotonsport hearts oak set ghana...,"[redford, vision, sundance, despite, sporting,..."
4,sir paul rocks super bowl crowds sir paul mcca...,"[mauresmo, open, victory, la, amelie, mauresmo..."
...,...,...
2995,uk directors guild nominees named martin scors...,"[steel, firm, cut, 45, 000, job, mittal, steel..."
2996,u2 play grammy awards show irish rock band u2 ...,"[israel, look, u, bank, chief, israel, asked, ..."
2997,pountney handed ban fine northampton coach bud...,"[india, iran, gas, export, deal, india, signed..."
2998,belle named best scottish band belle sebastian...,"[mido, make, third, apology, ahmed, mido, hoss..."


MODEL APPROACH
Word embeddings are low dimensional vectors obtained by training a neural network on a large corpus to predict a word given a context (Continuous Bag Of Words model) or to predict the context given a word (skip gram model). The context is a window of surrounding words. Pre-trained word embeddings are also available in the word2vec code.google page.

In this i am using Google news pre trained vectors and compare similarity between text1 & text2 using n_similarity method in gensim library which is nothing compares cosine similarity between two

Consider it as a unsupervised problem.

In [14]:
# Load pre_trained Google News Vectors after download file

wordmodelfile="/Users/shabnam/Downloads/DataNeuron_DataScience_Task1/untitled folder/GoogleNews-vectors-negative300.bin"
wordmodel= gensim.models.KeyedVectors.load_word2vec_format(wordmodelfile, binary=True)

In [15]:
wordmodel.similarity("king","queen")

0.6510956

In [16]:
cosine_similarity(wordmodel['good'].reshape(1,-1),wordmodel['bad'].reshape(1,-1))

array([[0.7190052]], dtype=float32)

In [17]:
final_list_text1 = df['text1'].to_list()
final_list_text2 = df['text2'].to_list()

In [18]:

def vectorizer(list_of_docs,model):
    
    feature =[] # save rew vector 
    for rew in list_of_docs: # iterating over reviews
        zero_vector = np.zeros(model.vector_size) # keyerror
        vectors =[] # to append vector of each word
        for word in rew : # iterating over all words in a review
            if word in model: # checking if word is there in our vocab
                try :
                    vectors.append(model[word]) # appending vector of each word
                except KeyError:
                    continue
        if vectors: # if Vectors is a empty list or not 
            vectors = np.asarray(vectors) # converting multiple arrays into a single array
            avg_vec = vectors.mean(axis=0) # avg of all vectors
            feature.append(avg_vec) # appending the avg vector
        else :
            feature.append(zero_vector) # handling key error
    return feature

In [19]:

vector_text1 = vectorizer(final_list_text1,wordmodel)
vector_text2 = vectorizer(final_list_text2,wordmodel)

In [20]:

similarity = []
for i in range(len(vector_text1)):
    text1 = vector_text1[i].reshape(1,-1)
    text2 = vector_text2[i].reshape(1,-1)
    score = cosine_similarity(text1,text2)[0][0]
    similarity.append(score)

In [21]:
df['similarity'] = similarity
df

,text1,text2,similarity
0,broadband challenges tv viewing number europea...,"[gardener, win, double, glasgow, britain, jaso...",0.451430
1,rap boss arrested drug find rap mogul marion s...,"[amnesty, chief, lament, war, failure, lack, p...",0.279069
2,player burn worries robinson england coach and...,"[hank, greeted, wintry, premiere, hollywood, s...",0.371034
3,hearts oak 3 2 cotonsport hearts oak set ghana...,"[redford, vision, sundance, despite, sporting,...",0.295228
4,sir paul rocks super bowl crowds sir paul mcca...,"[mauresmo, open, victory, la, amelie, mauresmo...",0.356393
...,...,...,...
2995,uk directors guild nominees named martin scors...,"[steel, firm, cut, 45, 000, job, mittal, steel...",0.291368
2996,u2 play grammy awards show irish rock band u2 ...,"[israel, look, u, bank, chief, israel, asked, ...",0.465628
2997,pountney handed ban fine northampton coach bud...,"[india, iran, gas, export, deal, india, signed...",0.342331
2998,belle named best scottish band belle sebastian...,"[mido, make, third, apology, ahmed, mido, hoss...",0.353406


In [ ]:
final_score.to_csv('final_score.csv',index=False)
